## Setup Agent

In [1]:
from langchain_community.agent_toolkits import SparkSQLToolkit, create_spark_sql_agent
from langchain_community.utilities.spark_sql import SparkSQL
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline

In [29]:
from pyspark.sql import SparkSession
import glob

spark = SparkSession.builder.getOrCreate()
schema = "gdelt"
table = "events"
try:
    spark.sql(f"DROP SCHEMA {schema} CASCADE")
except:
    pass
try:
    spark.sql(f"DROP TABLE {table}")
except:
    pass
spark.sql(f"CREATE DATABASE IF NOT EXISTS {schema}")
spark.sql(f"USE {schema}")
directory = "/home/tiago/factored-datathon-2024-seed42/files"

gkg_files = glob.glob(directory + '/**/*export.CSV', recursive=True)
csv_files = [file for file in gkg_files if file.lower().endswith('.csv')]
spark.read.csv(csv_files, sep='\t', header=True, inferSchema=True).write.saveAsTable(table)
spark.table(table).show()


24/08/17 17:26:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/08/17 17:26:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/08/17 17:26:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/08/17 17:26:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/08/17 17:26:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
24/08/17 17:26:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
24/08/17 17:26:36 WARN MemoryManager: Total allocation exceeds 95.

+-------------+--------+---------+----+------------+----------+-------------+-----------------+--------------------+----------------+-------------------+-------------------+---------------+---------------+---------------+----------+----------+-----------------+--------------------+----------------+-------------------+-------------------+---------------+---------------+---------------+-----------+---------+-------------+-------------+---------+--------------+-----------+----------+-----------+-----------------+--------------+--------------------+---------------------+------------------+-------------+--------------+-------------------+--------------+------------------+---------------------+------------------+-------------+--------------+-------------------+--------------+--------------------+---------------------+------------------+-------------+--------------+-------------------+---------+--------------------+
|GLOBALEVENTID| SQLDATE|MonthYear|Year|FractionDate|Actor1Code|   Actor1Name|

In [7]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your API token: ")

In [40]:
from langchain.chat_models import ChatOpenAI

spark_sql = SparkSQL(schema=schema)
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")
toolkit = SparkSQLToolkit(db=spark_sql, llm=llm)
agent_executor = create_spark_sql_agent(llm=llm, toolkit=toolkit, max_iterations=10, verbose=True)


In [48]:
print(agent_executor.invoke("The event codes 171, 172, 173, 174, 175 are related to sanctions. The QuadClass column represents different types of events (e.g., verbal cooperation, material conflict). Analyzing how these classes change over time can provide insights into the nature of international relations.")['output'])



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: 
Observation: events
Thought:The database contains a table named "events". I need to check the schema of this table to understand which columns are available and relevant for identifying actors involved in sanction-related events.

Action: schema_sql_db
Action Input: events
Observation: CREATE TABLE spark_catalog.gdelt.events (
  GLOBALEVENTID INT,
  SQLDATE INT,
  MonthYear INT,
  Year INT,
  FractionDate DOUBLE,
  Actor1Code STRING,
  Actor1Name STRING,
  Actor1CountryCode STRING,
  Actor1KnownGroupCode STRING,
  Actor1EthnicCode STRING,
  Actor1Religion1Code STRING,
  Actor1Religion2Code STRING,
  Actor1Type1Code STRING,
  Actor1Type2Code STRING,
  Actor1Type3Code STRING,
  Actor2Code STRING,
  Actor2Name STRING,
  Actor2CountryCode STRING,
  Actor2KnownGroupCode STRING,
  Actor2EthnicCode STRING,
  Actor2Religion1Code STRING,
  Actor2Religion2Code STRING,
  Actor2Type1Code STRING,
  Actor2Type2Code STR

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `The actors most frequently involved in sanction-related events (event codes 171, 172, 173, 174, 175) are as follows:

1. **UNITED STATES** (Actor1) - 730 occurrences
2. **UNITED STATES** (Actor2) - 439 occurrences
3. **PRISON** (Actor1) - 305 occurrences
4. **UNITED STATES** (both Actor1 and Actor2) - 299 occurrences
5. **POLICE** (Actor1) - 231 occurrences
6. **UNITED KINGDOM** (Actor1) - 156 occurrences
7. **UNITED STATES** (Actor1) and **PRISON** (Actor2) - 151 occurrences
8. **POLICE** (Actor1) and **UNITED STATES** (Actor2) - 151 occurrences
9. **POLICE** (Actor2) - 107 occurrences
10. **CRIMINAL** (Actor1) - 83 occurrences

These results show that the United States is the most frequently involved actor in sanction-related events, either as Actor1 or Actor2.`